In [1]:
import json
import random
from pathlib import Path

%load_ext autoreload
%autoreload 2
Path.ls = lambda x: list(x.iterdir())

In [2]:
# !pip install pymagnitude==0.1.117

In [3]:
# !pip install lz4 xxhash annoy fasteners torch

In [4]:
from pymagnitude import Magnitude
vectors = Magnitude("glove/heavy/glove.twitter.27B.200d.magnitude")

In [7]:
# vectors.most_similar_to_given("meghana", ["cat", "dog", "deadinside"])

In [ ]:
import string
from typing import Any, List

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageDraw, ImageFont

from wordzones.core import Paint

%matplotlib inline
matplotlib.rcParams["savefig.pad_inches"] = 0

In [ ]:
p = Paint()
p.show_canvas()

In [ ]:
def divide_chunks(l, n=3):
    # looping till length l
    for i in range(0, len(l), n):
        yield l[i : i + n]


def flatten(listOfLists: List[List[Any]]) -> List[Any]:
    return [item for sublist in listOfLists for item in sublist]

In [ ]:
def get_canvas(
    max_char_width,
    max_block_height,
    n_chunks,
    scale,
    word_pad_perc=0.1,
    line_pad_perc=0.3,
    background_color="white",
):
    w_pad, h_pad = word_pad_perc, line_pad_perc
    char_width, block_height = (
        int(max_char_width * (1 + w_pad)),
        int(max_block_height * (1 + h_pad)),
    )
    #     print(char_width, block_height)
    w, h = int(char_width * 3 * scale), int(block_height * n_chunks * scale)
    img = Image.new("RGB", (w, h), color=background_color)
    return img, char_width, int(block_height * 1.2)

In [ ]:
def canvas(
    zones: List[List[str]] = [
        ["assignment", "points", "test", "work", "expect"],
        ["molecules", "receptor", "binding", "protein"],
        ["india", "pakistan", "sri lanka"],
        ["usa", "mexico", "canada"],
        ["energy", "oil", "gas", "solar", "nuclear"],
        ["china", "vietnam", "indonesia", "thailand", "japan", "malaysia", "singapore"],
    ],
    hide_axes=False,
    font_sz=75,
    theme_colors: List[List[int]] = [
        [62, 98, 160],
        [247, 56, 56],
        [127, 76, 57],
        [65, 181, 136],
        [232, 176, 99],
        [66, 167, 181],
    ],
    scale=2,
    shuffle_zones: bool = False,
):

    assert len(theme_colors) >= 6
    font = ImageFont.truetype(font="DroidSans.ttf", size=font_sz * scale)
    w_font, h_font = font.getsize("gh")
    chunked_zones = list(divide_chunks(zones))
    n_chunks = len(chunked_zones)

    max_char_width, max_block_height = (
        len(max(flatten(zones), key=len)) * w_font,
        len(max(zones, key=len)) * h_font,
    )
    #     print(max_char_width, max_block_height, w_font, h_font)
    img, char_width, block_height = get_canvas(
        max_char_width, max_block_height, n_chunks, scale
    )
    if shuffle_zones:
        random.shuffle(chunked_zones)
    for block, zone in enumerate(chunked_zones):
        d = ImageDraw.Draw(img)
        for x, wordlist in enumerate(zone):
            n, c = len(wordlist), (x + 1) % 3
            text_colour = tuple(random.sample(range(0, 255), 3))
            if theme_colors:
                text_color = tuple(theme_colors.pop())
                print(f"Use {text_color} from theme colors")
            text_params = {"fill": text_color, "font": font}
            for y, word in enumerate(wordlist):
                line_pos = (h_font * y) + (max_block_height * block)
                d.text(
                    (10 + int(max_char_width * 1.1) * x * scale, 20 + line_pos),
                    word,
                    **text_params,
                )
    if hide_axes:
        ax = plt.axes([0, 0, 1, 1], frameon=False)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        ax.get_xaxis().set_visible(False)
    im_array = np.asarray(img)
    plt.imshow(im_array)
    plt.savefig("test.png", optimize=True)


canvas(hide_axes=True)